In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

# 1. Estimación de Insumos necesarios en el área ambulatoria

Dentro del área ambulatoria, se quieren estimar la cantidad de:

- Consultas Nuevas
- Consultas Repetidas
- Boxes
- Procedimientos Frecuentes (Tomografías, Resonancias, etc...)